# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [90]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [116]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [117]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Parameters:**

* **T:** Number of time steps (T = 10)
* **f(t):** Water inflow function at point A (f(t) = max(-4/30t^3+t^2-0.234t+3, 0))
* **C<sub>ij</sub>:** Capacity of pipe connecting points i and j (e.g., C<sub>AB</sub> = 3)
* **P:** Set of all pipes: {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}

**Decision Variables:**

* **x<sub>ijt</sub>:** Water flow through pipe ij at time step t (e.g., x<sub>AB1</sub> represents the flow from A to B at time step 1)
* **y<sub>ij</sub>:** Binary variable indicating whether pipe ij is activated (1) or deactivated (0)
* **r<sub>it</sub>:** Amount of water released at point i at time step t

**Objective Function:**

Maximize the total water outflow at point G over all time steps:

```
Maximize:  ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow Conservation:** At each point (except A and G) and each time step, the total inflow must equal the total outflow plus any released water:

    * For all i ∈ {B, C, D, E, F} and t ∈ {1,...,T}:  
        ∑<sub>j:(i,j)∈P</sub> x<sub>ijt</sub> + r<sub>it</sub> = ∑<sub>j:(j,i)∈P</sub> x<sub>jit-1</sub> 

2. **Inflow at Point A:** The inflow at point A is determined by the function f(t):

    * For t ∈ {1,...,T}: 
        ∑<sub>j:(A,j)∈P</sub> x<sub>Ajt</sub> = f(t)

3. **Outflow at Point G:**  All water reaching point G is considered output:

    * For t ∈ {1,...,T}: 
        ∑<sub>j:(j,G)∈P</sub> x<sub>jGt</sub> = x<sub>EGt</sub>

4. **Pipe Capacity:** The flow through each pipe cannot exceed its capacity:

    * For all (i,j) ∈ P and t ∈ {1,...,T}: 
        0 ≤ x<sub>ijt</sub> ≤ C<sub>ij</sub> * y<sub>ij</sub> 

5. **Pipe Activation:** Only 10 pipes can be activated:

    * ∑<sub>(i,j)∈P</sub> y<sub>ij</sub> = 10

6. **Non-negativity:** All flows and release amounts must be non-negative:

    * For all (i,j) ∈ P and t ∈ {1,...,T}:  x<sub>ijt</sub> ≥ 0
    * For all i ∈ {B, C, D, E, F} and t ∈ {1,...,T}:  r<sub>it</sub> ≥ 0

7. **Binary Activation:**

    * For all (i,j) ∈ P: y<sub>ij</sub> ∈ {0, 1}

This model combines elements of network flow optimization and integer programming. Solving this model will determine the optimal flow through the network, the activation status of each pipe, and the amount of water released at each point, maximizing the water reaching point G. 


## 3. Generate the pyomo code

In [118]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [119]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data 
T = 10 # Number of time steps
f = lambda t: max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) # Inflow function at point A
C = { # Capacity of pipes
    ('A','B'): 3, ('A','C'): 2, ('A','F'): 4, 
    ('B','C'): 1, ('B','D'): 2, ('B','E'): 3,
    ('C','E'): 2, ('C','F'): 1, 
    ('D','E'): 2, 
    ('E','G'): 5,
    ('F','B'): 1, ('F','G'): 3
}
P = list(C.keys()) # Set of all pipes

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T) # Time steps
model.P = pyo.Set(initialize=P) # Pipes
model.N = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E', 'F', 'G']) # All points
model.N_release = pyo.Set(initialize=['B', 'C', 'D', 'E', 'F']) # Points with release allowed

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipe ij at time t
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N_release, model.T, within=pyo.NonNegativeReals) # Release at point i at time t

# Objective Function
model.obj = pyo.Objective(expr=sum(model.x['E','G',t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Flow Conservation
def flow_conservation_rule(model, i, t):
    if t == 1:
        return sum(model.x[j,i,t] for j in model.N if (j,i) in model.P) == sum(model.x[i,j,t] for j in model.N if (i,j) in model.P) + model.r[i,t] 
    else:
        return sum(model.x[j,i,t] for j in model.N if (j,i) in model.P) + model.r[i,t] == sum(model.x[i,j,t-1] for j in model.N if (i,j) in model.P)  
model.flow_conservation = pyo.Constraint(model.N_release, model.T, rule=flow_conservation_rule)

# 2. Inflow at Point A
def inflow_A_rule(model, t):
    return sum(model.x['A',j,t] for j in model.N if ('A',j) in model.P) == f(t)
model.inflow_A = pyo.Constraint(model.T, rule=inflow_A_rule)

# 3. Outflow at Point G
def outflow_G_rule(model, t):
    return sum(model.x[j,'G',t] for j in model.N if (j,'G') in model.P) == model.x['E', 'G', t]
model.outflow_G = pyo.Constraint(model.T, rule=outflow_G_rule)

# 4. Pipe Capacity
def pipe_capacity_rule(model, i, j, t):
    return model.x[i,j,t] <= C[i,j] * model.y[i,j]
model.pipe_capacity = pyo.Constraint(model.P, model.T, rule=pipe_capacity_rule)

# 5. Pipe Activation
def pipe_activation_rule(model):
    return sum(model.y[i,j] for (i,j) in model.P) == 10
model.pipe_activation = pyo.Constraint(rule=pipe_activation_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Objective Function Value:", pyo.value(model.obj))
```

This code defines the problem in Pyomo, including sample data for the inflow function and pipe capacities. You can replace this sample data with your own data. The code defines the objective function, all the constraints, and finally solves the problem using the 'glpk' solver. 

This code assumes that a solver like 'glpk' is installed in your environment. 


## 4. Input problem data and try running the generated code

In [120]:
import pyomo.environ as pyo

# Sample Data 
T = 10 # Number of time steps
f = lambda t: max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) # Inflow function at point A
C = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}
P = list(C.keys()) # Set of all pipes

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T) # Time steps
model.P = pyo.Set(initialize=P) # Pipes
model.N = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E', 'F', 'G']) # All points
model.N_release = pyo.Set(initialize=['B', 'C', 'D', 'E', 'F']) # Points with release allowed

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipe ij at time t
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N_release, model.T, within=pyo.NonNegativeReals) # Release at point i at time t

# Objective Function
model.obj = pyo.Objective(expr=sum(model.x['E','G',t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Flow Conservation
def flow_conservation_rule(model, i, t):
    if t == 1:
        return sum(model.x[j,i,t] for j in model.N if (j,i) in model.P) == sum(model.x[i,j,t] for j in model.N if (i,j) in model.P) + model.r[i,t] 
    else:
        return sum(model.x[j,i,t] for j in model.N if (j,i) in model.P) + model.r[i,t] == sum(model.x[i,j,t-1] for j in model.N if (i,j) in model.P)  
model.flow_conservation = pyo.Constraint(model.N_release, model.T, rule=flow_conservation_rule)

# 2. Inflow at Point A
def inflow_A_rule(model, t):
    return sum(model.x['A',j,t] for j in model.N if ('A',j) in model.P) == f(t)
model.inflow_A = pyo.Constraint(model.T, rule=inflow_A_rule)

# 3. Outflow at Point G
def outflow_G_rule(model, t):
    return sum(model.x[j,'G',t] for j in model.N if (j,'G') in model.P) == model.x['E', 'G', t]
model.outflow_G = pyo.Constraint(model.T, rule=outflow_G_rule)

# 4. Pipe Capacity
def pipe_capacity_rule(model, i, j, t):
    return model.x[i,j,t] <= C[i,j] * model.y[i,j]
model.pipe_capacity = pyo.Constraint(model.P, model.T, rule=pipe_capacity_rule)

# 5. Pipe Activation
def pipe_activation_rule(model):
    return sum(model.y[i,j] for (i,j) in model.P) == 10
model.pipe_activation = pyo.Constraint(rule=pipe_activation_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Objective Function Value:", pyo.value(model.obj))


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 191
  Number of variables: 182
  Number of nonzeros: 532
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0390322208404541

ERROR: evaluating object as numeric value: x[E,G,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[E,G,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[E,G,1]


ValueError: No value for uninitialized NumericValue object x[E,G,1]

## 5. Correct the code to verify model viability (optional)

In [121]:
model.pprint()

3 Set Declarations
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'A', 'B', 'C', 'D', 'E', 'F', 'G'}
    N_release : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'B', 'C', 'D', 'E', 'F'}
    P : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   12 : {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}

1 RangeSet Declarations
    T : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

3 Var Declarations
    r : Size=50, Index=N_release*T
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
         ('B', 1) :     0 :  None :  None : False :  True : NonNegativeReals
         ('B', 2) :     0 :  None :  None

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [122]:
print(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Parameters:**

* **T:** Number of time steps (T = 10)
* **f(t):** Water inflow function at point A (f(t) = max(-4/30t^3+t^2-0.234t+3, 0))
* **C<sub>ij</sub>:** Capacity of pipe connecting points i and j (e.g., C<sub>AB</sub> = 3)
* **P:** Set of all pipes: {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}

**Decision Variables:**

* **x<sub>ijt</sub>:** Water flow through pipe ij at time step t (e.g., x<sub>AB1</sub> represents the flow from A to B at time step 1)
* **y<sub>ij</sub>:** Binary variable indicating whether pipe ij is activated (1) or deactivated (0)
* **r<sub>it</sub>:** Amount of water released at point i at time step t

**Objective Function:**

Maximize the total water outflow at point G over all time steps:

```
Maximize:  ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow Conservation:** At each point (except A and G) and each time step, the total inflow must equal the total ou

In [123]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data 
T = 10 # Number of time steps
f = lambda t: max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) # Inflow function at point A
C = { # Capacity of pipes
    ('A','B'): 3, ('A','C'): 2, ('A','F'): 4, 
    ('B','C'): 1, ('B','D'): 2, ('B','E'): 3,
    ('C','E'): 2, ('C','F'): 1, 
    ('D','E'): 2, 
    ('E','G'): 5,
    ('F','B'): 1, ('F','G'): 3
}
P = list(C.keys()) # Set of all pipes

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T) # Time steps
model.P = pyo.Set(initialize=P) # Pipes
model.N = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E', 'F', 'G']) # All points
model.N_release = pyo.Set(initialize=['B', 'C', 'D', 'E', 'F']) # Points with release allowed

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipe ij at time t
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N_release, model.T, within=pyo.NonNegativeReals) # Release at poi